In [2]:
import pandas as pd
import requests
import json
import xmltodict
import pickle

def bus_cnt(bus_no):
    info = pd.read_csv(r"C:\Users\0811m\Desktop\DScover\메인프로젝트\서울시노선기본정보_수정.csv")
    station_info = pd.read_csv(fr"C:\Users\0811m\Desktop\DScover\메인프로젝트\arsId_{bus_no}.csv")
    station_ars = list(station_info["arsId"].unique().astype(int).astype(str))

    api_dict = {}
    servicekey = 'Z%2FDKASzEwlIASKLfw%2FQQNTrexprISYo9aeuVrpI3dL2RyUpavjBPY6oJ5EPc5A6dE0XCA2HEURSO%2BOmu8lfw7w%3D%3D'
    for arsid in station_ars:
        def fetch_data(arsid):
            # API로부터 데이터 요청
            url = f'http://ws.bus.go.kr/api/rest/stationinfo/getRouteByStation?ServiceKey={servicekey}&arsId={arsid}'
            response = requests.get(url)
            contents = response.text

            # XML을 JSON으로 변경
            data_dict = xmltodict.parse(contents)
            return data_dict

        # 데이터 요청 시도
        data_dict = fetch_data(arsid)

        # 결과가 없는 경우, arsid의 앞자리에 0을 붙여서 다시 시도
        if data_dict['ServiceResult']['msgHeader']['headerMsg'] == '결과가 없습니다.':
            arsid = '0' + arsid
            data_dict = fetch_data(arsid)

        # itemList 추출
        try:
            item_list = data_dict['ServiceResult']['msgBody']['itemList']
        except (TypeError, KeyError):
            print(f"Error: Cannot extract itemList for station {arsid}")
            continue

        # 각 항목을 데이터프레임으로 변환하여 리스트에 추가
        dfs = []
        for item in item_list:
            df = pd.DataFrame([item])
            dfs.append(df)

        # 리스트에 있는 모든 데이터프레임을 합쳐서 하나의 데이터프레임으로 만듦
        combined_df = pd.concat(dfs, ignore_index=True)

        # 노선명을 키로 하여 딕셔너리에 추가
        api_dict[arsid] = combined_df


        with open(rf'C:\Users\0811m\Desktop\DScover\메인프로젝트\getRouteByStation_{bus_no}.pickle','wb') as fw:
            pickle.dump(api_dict, fw)


In [ ]:
bus_cnt("100")
bus_cnt("140")
bus_cnt("143")
bus_cnt("150")

In [3]:
bus_cnt("151")
bus_cnt("160")
bus_cnt("162")
bus_cnt("171")
bus_cnt("172")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_42776\3419434007.py:10: RuntimeWarning: invalid value encountered in cast
  station_ars = list(station_info["arsId"].unique().astype(int).astype(str))


Error: Cannot extract itemList for station 0-2147483648


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_42776\3419434007.py:10: RuntimeWarning: invalid value encountered in cast
  station_ars = list(station_info["arsId"].unique().astype(int).astype(str))


Error: Cannot extract itemList for station 0-2147483648


In [5]:
import pickle
import pandas as pd

bus_numbers = ["100", "140", "143", "150", "151", "160", "162", "171", "172"]

# 각 노선에 대해 처리
for bus_no in bus_numbers:
    # pickle 파일 로드
    file_path = f'C:\\Users\\0811m\\Desktop\\DScover\\메인프로젝트\\getRouteByStation_{bus_no}.pickle'
    with open(file_path, 'rb') as handle:
        getRouteByStation = pickle.load(handle)

    # 결과를 저장할 리스트 초기화
    result_list = []

    # 각 arsId에 대해 busRouteAbrv의 unique 값을 count
    for arsid, df in getRouteByStation.items():
        bus_count = df.shape[0]
        result_list.append({'arsId': arsid, 'bus_cnt': bus_count})

    # 리스트를 데이터프레임으로 변환
    result_df = pd.DataFrame(result_list)

    # 노선별 결과를 CSV 파일로 저장
    output_file = f'C:\\Users\\0811m\\Desktop\\DScover\\메인프로젝트\\bus_{bus_no}_route_counts.csv'
    result_df.to_csv(output_file, index=False)
